# Cobify: fuel consumption election

The aim of this analysis is to analyze fuel data in order to find the best combustible for Cobify.

#### Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
measurements_df = pd.read_csv('data/measurements.csv', )
print('Shape of dataframe:', measurements_df.shape)
measurements_df.head(10)

Shape of dataframe: (388, 12)


,distance,consume,speed,temp_inside,temp_outside,specials,gas_type,AC,rain,sun,refill liters,refill gas
0,28,5,26,"21,5",12,NaN,E10,0,0,0,45,E10
1,12,"4,2",30,"21,5",13,NaN,E10,0,0,0,NaN,NaN
2,"11,2","5,5",38,"21,5",15,NaN,E10,0,0,0,NaN,NaN
3,"12,9","3,9",36,"21,5",14,NaN,E10,0,0,0,NaN,NaN
4,"18,5","4,5",46,"21,5",15,NaN,E10,0,0,0,NaN,NaN
5,"8,3","6,4",50,"21,5",10,NaN,E10,0,0,0,NaN,NaN
6,"7,8","4,4",43,"21,5",11,NaN,E10,0,0,0,NaN,NaN
7,"12,3",5,40,"21,5",6,NaN,E10,0,0,0,NaN,NaN
8,"4,9","6,4",26,"21,5",4,NaN,E10,0,0,0,NaN,NaN
9,"11,9","5,3",30,"21,5",9,NaN,E10,0,0,0,NaN,NaN


In [3]:
print('SUMMARY OF NUMERIC COLUMNS')
measurements_df.describe()

SUMMARY OF NUMERIC COLUMNS


,speed,temp_outside,AC,rain,sun
count,388.000000,388.000000,388.000000,388.000000,388.000000
mean,41.927835,11.358247,0.077320,0.123711,0.082474
std,13.598524,6.991542,0.267443,0.329677,0.275441
min,14.000000,-5.000000,0.000000,0.000000,0.000000
25%,32.750000,7.000000,0.000000,0.000000,0.000000
50%,40.500000,10.000000,0.000000,0.000000,0.000000
75%,50.000000,16.000000,0.000000,0.000000,0.000000
max,90.000000,31.000000,1.000000,1.000000,1.000000


In [4]:
print('SUMMARY OF CATEGORICAL COLUMNS')
measurements_df.describe(exclude=np.number)

SUMMARY OF CATEGORICAL COLUMNS


,distance,consume,temp_inside,specials,gas_type,refill liters,refill gas
count,388,388,376,93,388,13,13
unique,174,43,13,12,2,10,2
top,"11,8",5,"21,5",rain,SP98,45,SP98
freq,37,27,133,32,228,2,8


From the summaries above, we can observe some irregularities:

- Columns consume, temp_inside, distance as refill liters should be of numerical type, not categorical.
- Columns refill liters and refill gas too many NaN values (76%). It would be wise to drop those columns as they don't not give us any valuable information.
- Column specials contains repetead information (it is a string specifying some of these values: AC, rain, sun). We can drop it.

#### Changing the columns type

In [5]:
measurements_df['distance'] = measurements_df['distance'].apply(lambda x: x.replace(',', '.'))
measurements_df['consume'] = measurements_df['consume'].apply(lambda x: x.replace(',', '.'))

In [6]:
measurements_df['distance'] = measurements_df['distance'].apply(pd.to_numeric)
measurements_df['consume'] = measurements_df['consume'].apply(pd.to_numeric)

In [7]:
def from_str_to_num(i):
    if type(i) == str:
        if "," in i:
            i_dot = i.replace(",", '.')
            i_num = float(i_dot)
            return i_num
        else:
            i_num = float(i)
            return i_num

In [8]:
measurements_df['temp_inside'] = measurements_df['temp_inside'].apply(from_str_to_num)

Checking df types...

In [9]:
measurements_df.dtypes

distance         float64
consume          float64
speed              int64
temp_inside      float64
temp_outside       int64
specials          object
gas_type          object
AC                 int64
rain               int64
sun                int64
refill liters     object
refill gas        object
dtype: object

In [10]:
measurements_df.describe()

,distance,consume,speed,temp_inside,temp_outside,AC,rain,sun
count,388.000000,388.000000,388.000000,376.000000,388.000000,388.000000,388.000000,388.000000
mean,19.652835,4.912371,41.927835,21.929521,11.358247,0.077320,0.123711,0.082474
std,22.667837,1.033172,13.598524,1.010455,6.991542,0.267443,0.329677,0.275441
min,1.300000,3.300000,14.000000,19.000000,-5.000000,0.000000,0.000000,0.000000
25%,11.800000,4.300000,32.750000,21.500000,7.000000,0.000000,0.000000,0.000000
50%,14.600000,4.700000,40.500000,22.000000,10.000000,0.000000,0.000000,0.000000
75%,19.000000,5.300000,50.000000,22.500000,16.000000,0.000000,0.000000,0.000000
max,216.100000,12.200000,90.000000,25.500000,31.000000,1.000000,1.000000,1.000000


Droping columns refill liters and refill gas...

In [11]:
measurements_df = measurements_df.drop(['refill liters', 'refill gas'], axis = 1)
measurements_df.head()

,distance,consume,speed,temp_inside,temp_outside,specials,gas_type,AC,rain,sun
0,28.0,5.0,26,21.5,12,NaN,E10,0,0,0
1,12.0,4.2,30,21.5,13,NaN,E10,0,0,0
2,11.2,5.5,38,21.5,15,NaN,E10,0,0,0
3,12.9,3.9,36,21.5,14,NaN,E10,0,0,0
4,18.5,4.5,46,21.5,15,NaN,E10,0,0,0


In [12]:
for col in measurements_df.columns:
    print('Value counts of', col)
    print(measurements_df[col].value_counts())
    print('')

Value counts of distance
11.8    37
16.0    24
12.3    23
12.4    18
15.4    14
        ..
44.9     1
36.6     1
9.9      1
16.8     1
14.7     1
Name: distance, Length: 174, dtype: int64

Value counts of consume
5.0     27
4.5     27
4.6     22
4.7     21
4.3     20
4.1     20
5.1     19
4.8     19
4.4     17
4.0     17
4.9     16
4.2     15
5.3     14
3.9     14
5.2     12
3.8     11
5.7     11
5.6     10
5.4      9
3.7      8
5.8      7
5.5      6
5.9      5
3.6      4
6.4      4
6.1      4
6.2      4
6.0      4
6.3      4
8.1      2
6.5      2
7.4      2
6.9      1
8.7      1
12.2     1
7.9      1
10.8     1
9.9      1
7.1      1
3.3      1
9.0      1
11.5     1
6.6      1
Name: consume, dtype: int64

Value counts of speed
42    18
43    17
38    16
26    15
36    15
33    15
40    15
32    12
44    12
37    12
29    11
39    11
55    11
46    11
45    10
58    10
25    10
24     9
50     9
35     9
57     8
41     8
52     7
53     6
48     6
28     6
30     6
21     6
61     6
51

In [13]:
measurements_df.isna().sum()

distance          0
consume           0
speed             0
temp_inside      12
temp_outside      0
specials        295
gas_type          0
AC                0
rain              0
sun               0
dtype: int64

From the specials column, we can see that there are a lot of nan values. Also, the information that this column contains is, in some way, redudant. There are a small number of events where the specials column indicates that it's snowing, but we don't have enough information to consider it relevant. Therefore, we'll drop this column.

In [14]:
measurements_df = measurements_df.drop('specials', axis = 1)
measurements_df.head()

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun
0,28.0,5.0,26,21.5,12,E10,0,0,0
1,12.0,4.2,30,21.5,13,E10,0,0,0
2,11.2,5.5,38,21.5,15,E10,0,0,0
3,12.9,3.9,36,21.5,14,E10,0,0,0
4,18.5,4.5,46,21.5,15,E10,0,0,0


Last but not leasst, we can delete the rows with null values from temp_inside as the volum of null values is very small.

In [15]:
temp_nan_values = measurements_df[measurements_df['temp_inside'].isna()].index
temp_nan_values

Int64Index([93, 95, 97, 98, 99, 100, 102, 201, 203, 261, 267, 268], dtype='int64')

In [16]:
measurements_df = measurements_df.drop(index=temp_nan_values)
measurements_df.shape

(376, 9)

## How much does each trip cost us?

Although we have the fuel price info from Kaggle, fuel prices are highly volatile. Therefore, let's check the fuel prices for today in Madrid for E10 and SP98:
 - E10 = 1.77€
 - SP98 = 1.93€
 
Let's try to calculate how much did each journey cost.

In [17]:
measurements_df['fuel_price_liter'] = measurements_df['gas_type'].apply(lambda x: 1.77 if x == 'E10' else 1.93)
measurements_df['fuel_consume_km'] = (measurements_df['consume']*measurements_df['distance'])/100
measurements_df.head()

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,fuel_price_liter,fuel_consume_km
0,28.0,5.0,26,21.5,12,E10,0,0,0,1.77,1.4000
1,12.0,4.2,30,21.5,13,E10,0,0,0,1.77,0.5040
2,11.2,5.5,38,21.5,15,E10,0,0,0,1.77,0.6160
3,12.9,3.9,36,21.5,14,E10,0,0,0,1.77,0.5031
4,18.5,4.5,46,21.5,15,E10,0,0,0,1.77,0.8325


In [18]:
measurements_df['trip_cost'] = measurements_df['fuel_consume_km']*measurements_df['fuel_price_liter']
measurements_df['cost_km'] = measurements_df['trip_cost']/measurements_df['distance']
measurements_df.head()

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,fuel_price_liter,fuel_consume_km,trip_cost,cost_km
0,28.0,5.0,26,21.5,12,E10,0,0,0,1.77,1.4000,2.478000,0.08850
1,12.0,4.2,30,21.5,13,E10,0,0,0,1.77,0.5040,0.892080,0.07434
2,11.2,5.5,38,21.5,15,E10,0,0,0,1.77,0.6160,1.090320,0.09735
3,12.9,3.9,36,21.5,14,E10,0,0,0,1.77,0.5031,0.890487,0.06903
4,18.5,4.5,46,21.5,15,E10,0,0,0,1.77,0.8325,1.473525,0.07965


In [19]:
measurements_df.groupby('gas_type').agg({'cost_km':'mean', 'consume':'mean'})

,cost_km,consume
gas_type,,
E10,0.087632,4.950955
SP98,0.094702,4.906849


It looks like E10 is the best solution in terms of cost-effectivess, althought its consume is higher than SP98. Let's save the file to create some visualizations in Tableau.

In [20]:
measurements_df.to_csv('data/measurements_clean.csv')